In [3]:
import pandas as pd
from pprint import pprint
import glob
import pandas as pd
import random
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from config import event_types
random.seed(42)
pd.set_option('display.max_rows', None)
# jupyter nbconvert --to script 04b_combine_results_find_disagreement.ipynb
# sudo kill -9 $(nvidia-smi | awk 'NR>8 {print $5}' | grep -E '^[0-9]+$')
version = 2
from config import event_types

def prepare_df(df, type="biolord"):
    global event_types
    df = df.copy()
    print("Loaded file len: ",len(df))
    df.columns = ['Document' if i=='DOCUMENT' else i for i in df.columns]
    df['Sent_ID'] = df['Events'].apply(lambda x: [f"{i:04d}" for i in range(len(x))])
    df = df.explode(["Sent_ID","Events"])
    print("After exploding file len: ",len(df))
    df['UID'] = df['SUBJECT_ID'].astype(str) + "_" + df['ROW_ID'].astype(str) + "_" + df['Sent_ID'].astype(str)
    df = df.dropna(subset="Events")
    df['Event_Name'] = df['Events'].apply(lambda x: x['event'])
    df['Sentence'] = df['Events'].apply(lambda x: x['text'])
    df['Time'] = df['Events'].apply(lambda x: x['event_detection_time'])
    
    if type == "dictionary":
        df['Keyword'] = df['Events'].apply(lambda x: x['keyword'])
        df['Lemma'] = df['Events'].apply(lambda x: x['lemma'])
        df['Keyword_Position'] = df['Events'].apply(lambda x: x['keyword_position'])
        
    if type == "biolord":
        df['Similarity'] = df['Events'].apply(lambda x: x['similarity'])
        df['Similarity'] = df['Similarity'].apply(lambda x: {k:v for (k,v) in x.items() if k!="Alert And Oriented"})
        for ET in event_types:
            df[f"{ET}_similarity"] = df['Similarity'].apply(lambda x:x[ET])
            
    return df
df_dictionary = prepare_df(pd.read_pickle(f"../exports/03b_selected_reports_with_event_log_only_dictionary_v{version}/combined.pkl"),type="dictionary")
# df_biolord = prepare_df(pd.read_pickle(f"../exports/selected_reports_with_event_log_only_biolord_v{version}/combined.pkl"), type = "biolord")
df_dictionary.to_pickle("../exports/04b_dictionary_features.pkl")





Loaded file len:  12524
After exploding file len:  280066


In [6]:
df_dictionary.Event_Name.explode().value_counts()

Event_Name
Unknown      256636
Pain           9281
Family         7577
Excretion      4910
Sleep          2558
Eating         1740
Name: count, dtype: int64

In [ ]:
N1 = 10
def combine_lists(x):
    combined = []
    for item in x:
        if item:
            if isinstance(item, (list, tuple)):
                combined.extend(item)
            else:
                combined.append(item)
    combined = [i for i in combined if i]
    res = set(combined)
    return list(res)
from config import event_types
import os
os.makedirs("../exports/04b_groundtruth/P-SET/Generated", exist_ok=True)
for ET in event_types:
    df_dictionary[f'Event_Name_{ET}'] = df_dictionary['Event_Name'].apply(lambda x: ET in x)
    row_id_to_doc_ET = df_dictionary.groupby('ROW_ID')[f'Event_Name_{ET}'].max().to_dict()
    df_dictionary[f'Event_Name_{ET}_doc'] = df_dictionary['ROW_ID'].apply(lambda x: row_id_to_doc_ET[x])
    people_ranked_by_ET = df_dictionary.groupby('SUBJECT_ID')[f'Event_Name_{ET}'].sum().sort_values(ascending=False).index.tolist()
    df_dictionary_topN1 = df_dictionary[df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]
    sentences_w_et_topn1 = df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}']]
    df_dictionary_rest = df_dictionary[~df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]
    sentences_w_et_rest = df_dictionary_rest[df_dictionary_rest[f'Event_Name_{ET}']].sample(len(sentences_w_et_topn1), random_state=42)
    Sentences_with_ET = pd.concat([sentences_w_et_topn1, sentences_w_et_rest]).reset_index(drop=True)
    Sentences_with_ET['is_keyword_present'] = True
    # find N2 such that len(df_dictionary[~df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N1])]) <= len(Sentences_with_ET)
    # N2 = N1
    # while len(df_dictionary[(df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N2])) & ~df_dictionary[f'Event_Name_{ET}']]) >= len(Sentences_with_ET):
    #     N2 -= 1
    # print(N2, len(df_dictionary[(df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N2])) & ~df_dictionary[f'Event_Name_{ET}']]), len(Sentences_with_ET))
    # df_dictionary_topN2 = df_dictionary[df_dictionary['SUBJECT_ID'].isin(people_ranked_by_ET[:N2])]
    Sentences_without_ET = (df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}']==False]
                            .sample(len(Sentences_with_ET), random_state=42))
    Sentences_without_ET['is_keyword_present'] = False
    Sentences = pd.concat([Sentences_with_ET, Sentences_without_ET]).reset_index(drop=True)
    Documents_with_ET = df_dictionary_topN1[df_dictionary_topN1[f'Event_Name_{ET}_doc']]
    Documents_with_ET['is_keyword_present'] = True
    Documents_without_ET =  df_dictionary_topN1[~df_dictionary_topN1[f'Event_Name_{ET}_doc']]
    Documents_without_ET['is_keyword_present'] = False
    Documents = pd.concat([Documents_with_ET, Documents_without_ET]).reset_index(drop=True)
    Documents['Document'] = Documents['Document'].apply(lambda x: x.lower())
    Documents = Documents.groupby(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'AGE', 'LOS_DAYS', 'IS_ALIVE', 'DURATION_NOTE'])[["Event_Name","Keyword","Document","Lemma"]].agg(lambda x:combine_lists(x)).reset_index()
    Documents['Document'] = [i[0] for i in Documents['Document']]
    print(f"For event type {ET}, N1={N1}, Sentences_with_ET={len(Sentences_with_ET)}, Sentences_without_ET={len(Sentences_without_ET)}, Sentences={len(Sentences)},Documents_with_ET={Documents_with_ET.ROW_ID.nunique()}, Documents_without_ET={Documents_without_ET.ROW_ID.nunique()}, Documents={Documents.ROW_ID.nunique()}")
    
    if os.path.exists(f"../exports/04_groundtruth/P-SET/Annotated/{ET}_Sentences.pkl"):
        old_annotations = pd.read_pickle(f"../exports/04_groundtruth/P-SET/Annotated/{ET}_Sentences.pkl")
        sent_to_gt = old_annotations.groupby(['Sentence'])[f'Sent_gt_{ET}'].max().to_dict()
        sent_to_negation = old_annotations.groupby(['Sentence'])[f'negation'].max().to_dict()
        sent_to_good_example = old_annotations.groupby(['Sentence'])[f'good_example'].max().to_dict()
        sent_to_comment = old_annotations.groupby(['Sentence'])[f'comment'].max().to_dict()
        Sentences[f'Sent_gt_{ET}'] = Sentences.Sentence.map(sent_to_gt)
        Sentences['negation'] = Sentences.Sentence.map(sent_to_negation)
        Sentences['good_example'] = Sentences.Sentence.map(sent_to_good_example)
        Sentences['comment'] = Sentences.Sentence.map(sent_to_comment)
    
    if os.path.exists(f"../exports/04_groundtruth/P-SET/Annotated/{ET}_Documents.pkl"):
        old_annotations = pd.read_pickle(f"../exports/04_groundtruth/P-SET/Annotated/{ET}_Documents.pkl")
        row_id_to_gt = old_annotations.groupby(['ROW_ID'])[f'Doc_gt_{ET}'].max().to_dict()
        row_id_to_negation = old_annotations.groupby(['ROW_ID'])[f'negation'].max().to_dict()
        row_id_to_good_example = old_annotations.groupby(['ROW_ID'])[f'good_example'].max().to_dict()
        row_id_to_comment = old_annotations.groupby(['ROW_ID'])[f'comment'].max().to_dict()
        Documents[f'Doc_gt_{ET}'] = Documents.ROW_ID.map(row_id_to_gt)
        Documents['negation'] = Documents.ROW_ID.map(row_id_to_negation)
        Documents['good_example'] = Documents.ROW_ID.map(row_id_to_good_example)
        Documents['comment'] = Documents.ROW_ID.map(row_id_to_comment)
        

    # Sentences.to_pickle(f"../exports/04b_groundtruth/P-SET/Generated/{ET}_Sentences.pkl")
    # Documents.to_pickle(f"../exports/04b_groundtruth/P-SET/Generated/{ET}_Documents.pkl")

/tmp/ipykernel_836966/1907072688.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_836966/1907072688.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_without_ET['is_keyword_present'] = False


For event type Pain, N1=10, Sentences_with_ET=734, Sentences_without_ET=734, Sentences=1468,Documents_with_ET=126, Documents_without_ET=49, Documents=175
1468 175


/tmp/ipykernel_836966/1907072688.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_836966/1907072688.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_without_ET['is_keyword_present'] = False


For event type Sleep, N1=10, Sentences_with_ET=300, Sentences_without_ET=300, Sentences=600,Documents_with_ET=83, Documents_without_ET=119, Documents=202
600 202


/tmp/ipykernel_836966/1907072688.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_836966/1907072688.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_without_ET['is_keyword_present'] = False


For event type Excretion, N1=10, Sentences_with_ET=412, Sentences_without_ET=412, Sentences=824,Documents_with_ET=153, Documents_without_ET=205, Documents=358
824 358


/tmp/ipykernel_836966/1907072688.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_836966/1907072688.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_without_ET['is_keyword_present'] = False


For event type Eating, N1=10, Sentences_with_ET=274, Sentences_without_ET=274, Sentences=548,Documents_with_ET=95, Documents_without_ET=246, Documents=341
548 341
For event type Family, N1=10, Sentences_with_ET=838, Sentences_without_ET=838, Sentences=1676,Documents_with_ET=176, Documents_without_ET=141, Documents=317
1676 317


/tmp/ipykernel_836966/1907072688.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_with_ET['is_keyword_present'] = True
/tmp/ipykernel_836966/1907072688.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Documents_without_ET['is_keyword_present'] = False


In [23]:
old_annotations.columns

Index(['ROW_ID', 'Event_Name', 'Keyword', 'Document', 'Lemma', 'Doc_gt_Sleep',
       'negation', 'good_example', 'comment'],
      dtype='object')

In [17]:
ET = "Sleep"
old_annotations = pd.read_pickle(f"../exports/04_groundtruth/P-SET/Annotated/{ET}_Documents.pkl")
new_annotations = pd.read_pickle(f"../exports/04b_groundtruth/P-SET/Generated/{ET}_Documents.pkl")

sent_to_gt = old_annotations.groupby('Document')[f'Doc_gt_{ET}'].max().to_dict()
len([i for i in new_annotations.Document.tolist() if i in old_annotations.Document.tolist()])

80

In [14]:
new_annotations.Sentence.tolist()

['.sleeping',
 'in naps.family at bedside most of day',
 'slept at intervals',
 'sister remained at bedside, sleeping on cot',
 'neuro: lethargic/sleeping @ change of shift',
 'becoming more interactive as night went on, but slept well through night',
 'patient sleeping in naps p med for pain.husband at bedside all day ',
 "feeling anxious and i can't sleep",
 'patient received ativan for anxiety, patient able to sleep but frequently waking up',
 'off to sleep, with -15sec of apnea associated with desat to 87%',
 'patient placed on 4l nc to sleep',
 'eval sleep apnea',
 'patient has complained that she hasnt slept in days',
 'she has finally been sleeping since line was pulled',
 '[**name (ni) **] husband will sleep here tonight',
 's:"i didn\'t sleep last night"',
 "sats 95 when awake, dropped to high 80's when sleeping",
 'patient dozing intermittently throughout day',
 's: "my son doesn\'t help at all at home, all he does is eat and sleep"',
 "[**name (ni) 591**] hypotensive slightl

In [15]:
len(df_dictionary_topN2[~df_dictionary_topN2[f'Event_Name_{ET}']]), len(Sentences_with_ET)

(355, 781)

In [18]:
Sentences.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME',
       'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'Document',
       'AGE', 'LOS_DAYS', 'IS_ALIVE', 'DOCUMENT_LOWER', 'NUM_NOTES',
       'MAX_NOTES_PER_DAY', 'MAX_NOTES_PER_CHARTTIME', 'DURATION_NOTE',
       'all_dates_present', 'Sentences', 'Sentences_Raw', 'Sentences_Cleaned',
       'Events', 'Sent_ID', 'UID', 'Event_Name', 'Sentence', 'Time', 'Keyword',
       'Lemma', 'Keyword_Position', 'Event_Name_Pain', 'Event_Name_Pain_doc',
       'Event_Name_Sleep', 'Event_Name_Sleep_doc', 'Event_Name_Excretion',
       'Event_Name_Excretion_doc', 'Event_Name_Eating',
       'Event_Name_Eating_doc', 'Event_Name_Family', 'Event_Name_Family_doc',
       'is_keyword_present'],
      dtype='object')

In [2]:
Documents.Document.iloc[0]

NameError: name 'Documents' is not defined

In [ ]:
N = 10
from config import event_types
import os
os.makedirs("../exports/04_groundtruth/P-SET/Generated", exist_ok=True)
df_dictionary_exploded = df_dictionary.explode("Event_Name")
for ET in event_types:
    top_ET = df_dictionary_exploded[df_dictionary_exploded.Event_Name==ET][["SUBJECT_ID","Event_Name"]].value_counts().reset_index().iloc[:N]
    selected_patient_sentences = df_dictionary[df_dictionary.SUBJECT_ID.isin(top_ET['SUBJECT_ID'].tolist())]
    total_ET = top_ET[top_ET.Event_Name==ET]['count'].sum()
    num_sentences = len(selected_patient_sentences.UID.unique())
    num_reports = len(selected_patient_sentences.ROW_ID.unique())
    num_patients = len(selected_patient_sentences.SUBJECT_ID.unique())
    print(f"***{ET}***",
          f"{ET} counts:{total_ET}\n",
        # f"Event counts:\n{selected_patient_sentences.Event_Name.value_counts()}\n"
        f"Unique Sentences: {num_sentences}\n",
        f"Unique Reports: {num_reports}\n",
        f"Unique Patients: {num_patients}\n"
    )
    
    
    # selected_patient_sentences.to_pickle(f"../exports/04_groundtruth/P-SET/Generated/04_{ET}_{num_patients}_{num_reports}_{num_sentences}_{total_ET}.pkl")

***Pain*** Pain counts:410
 Unique Sentences: 5207
 Unique Reports: 194
 Unique Patients: 10

***Sleep*** Sleep counts:166
 Unique Sentences: 4644
 Unique Reports: 196
 Unique Patients: 10

***Excretion*** Excretion counts:215
 Unique Sentences: 8557
 Unique Reports: 358
 Unique Patients: 10

***Eating*** Eating counts:148
 Unique Sentences: 6478
 Unique Reports: 303
 Unique Patients: 10

***Family*** Family counts:512
 Unique Sentences: 7515
 Unique Reports: 302
 Unique Patients: 10



In [13]:
df_dictionary_exploded.Event_Name.value_counts()

Event_Name
Unknown      256636
Pain           9281
Family         7577
Excretion      4910
Sleep          2558
Eating         1740
Name: count, dtype: int64

In [4]:
import pandas as pd
df = pd.read_pickle(f"../exports/04_dictionary_features.pkl")
ET = 'Sleep'
df_sentence_with_sleep = (df[df.Event_Name.apply(lambda x: ET in x)])
df_sentence_without_sleep = (df[df.Event_Name.apply(lambda x: ET not in x)])
top_10_patients = df_sentence_with_sleep.SUBJECT_ID.value_counts().head(10).index.tolist()
all_patients = pd.read_pickle("../exports/02b_filtered_patient_reports_7_14_days.pkl")['SUBJECT_ID'].unique().tolist()
len(top_10_patients), len(set(top_10_patients).intersection(set(all_patients)))
# df_top_10_patient_sentences_with_sleep = df_sentence_with_sleep[df_sentence_with_sleep.SUBJECT_ID.isin(top_10_patients)]
# df_top_10_patient_sentences_without_sleep = df_sentence_without_sleep[df_sentence_without_sleep.SUBJECT_ID.isin(top_10_patients)]
# df_not_top_10_patient_sentences = df_sentence_with_sleep[~df_sentence_with_sleep.SUBJECT_ID.isin(top_10_patients)]
# print("top 10 patient sentences:", len(df_top_10_patient_sentences_with_sleep))
# print("top 10 patients sentences without sleep:", len(df_top_10_patient_sentences_without_sleep))
# print("other patient sentences:", len(df_not_top_10_patient_sentences))

(10, 10)

In [4]:
df_top_10_patient_sentences_without_sleep.SUBJECT_ID.value_counts()

SUBJECT_ID
15883    749
23082    604
16459    541
22673    522
4874     489
25328    473
14995    453
3748     361
32291    271
7042     149
Name: count, dtype: int64

In [19]:
docid_isevent = df[df.SUBJECT_ID.isin(top_10_patients)].groupby(['ROW_ID','SUBJECT_ID'])['Event_Name'].agg(lambda x: any([ET in j for j in x])).reset_index()
docid_isevent[docid_isevent.Event_Name==True]['SUBJECT_ID'].value_counts()
res = docid_isevent.groupby(["Event_Name", "SUBJECT_ID"]).size().unstack(fill_value=0)
res, res.sum(axis=1)

(SUBJECT_ID  3748   4874   7042   14995  15883  16459  22673  23082  25328  \
 Event_Name                                                                  
 False           7      7      6      4     28     18     12     24     10   
 True            7      9      5     12      8      5     10      8      8   
 
 SUBJECT_ID  32291  
 Event_Name         
 False           5  
 True            9  ,
 Event_Name
 False    121
 True      81
 dtype: int64)

In [20]:
sentid_isevent = df[df.SUBJECT_ID.isin(top_10_patients)].groupby(['UID','SUBJECT_ID'])['Event_Name'].agg(lambda x: any([ET in j for j in x])).reset_index()
sentid_isevent[sentid_isevent.Event_Name==True]['SUBJECT_ID'].value_counts()
res = sentid_isevent.groupby(["Event_Name", "SUBJECT_ID"]).size().unstack(fill_value=0)
res, res.sum(axis=1)

(SUBJECT_ID  3748   4874   7042   14995  15883  16459  22673  23082  25328  \
 Event_Name                                                                  
 False         361    489    149    453    749    541    522    604    473   
 True           15     17     12     24     13     14     15     12     12   
 
 SUBJECT_ID  32291  
 Event_Name         
 False         271  
 True           13  ,
 Event_Name
 False    4612
 True      147
 dtype: int64)

In [50]:
for N2 in range(7):
    res = df_sentence_without_sleep[df_sentence_without_sleep.SUBJECT_ID.isin(top_10_patients[:N2])]
    print(N2,len(res))
    
2366 + 2502, 121 + 81

0 0
1 453
2 942
3 1464
4 1825
5 2366
6 2637


(4868, 202)

In [49]:
df[df.SUBJECT_ID.isin(top_10_patients)].ROW_ID.nunique()

202

In [35]:
2355 + 147

2502

df.LOS